In [2]:
import lumipy as lm
from datetime import datetime
from lusidjam import RefreshingToken as rt

# Tutorial 3 - Basic Queries

## Introduction: lumipy fluent syntax and the role of the atlas

Idea of lumipy's fluent expression syntax is to make it easy to build python code for building up luminesce queries.  

Allows you to construct a graph of operations using python operators and primitives that defines a query. It also offers support for tab completion and inspection of docstrings/method args with shift+tab in jupyter. 

The starting point is always the atlas. You take the provider attribute from the atlas and (optionally) supply parameters to it. 
```
table = atlas.some_provider(param=173)
```
This is the data table that you will chain methods from for SQL keywords such as `select` and `where`.  
```
qry = table.select('*').where(table.col > 0) 
```
Once built the query is sent off to `luminesce` by calling the `.go()` method. 
```
df = qry.go()
```
Detailed examples are given below. 

## Get the Atlas

In [5]:
atlas = lm.get_atlas('/Users/work/.lumi_secrets_qa.json')

## Get the Provider Class
We first get the holdings provider definition object from the atlas

In [3]:
atlas.lusid_portfolio_holding

└•Provider Definition: lusid_portfolio_holding
   ├•Table SQL Name: Lusid.Portfolio.Holding
   ├•Type: DataProvider
   ├•Category: Lusid
   ├•Description: Returns Lusid Portfolio / Holdings data
   ├•Documentation: [Not Available]
   ├•Last Ping At: 2021-06-13 14:46:50.179
   └•Fields:
      ├•as_at                          Parameter   DateTime  
      ├•cost_amount                    Column      Decimal   
      ├•cost_amount_portfolio_currency Column      Decimal   
      ├•cost_currency                  Column      Text      
      ├•effective_at                   Parameter   DateTime  
      ├•error                          Column      Text      
      ├•holding_prop                   Column      Text      
      ├•holding_type                   Column      Text      
      ├•inst_prop                      Column      Decimal   
      ├•instrument_tag                 Column      Text      
      ├•lusid_instrument_id            Column      Text      
      ├•portfolio_code         

Try doing `shift`+`tab` with the cursor inside the `atlas.lusid_portfolio_holding()` call below. 

In [4]:
holdings = atlas.lusid_portfolio_holding(
    effective_at=datetime(2021, 3, 1), 
    as_at=datetime(2021, 3, 8)
)
# Show holdings DAG
holdings

└•Table
   └•[DEFINE PROVIDER TABLE]
      ├•Parameter ⬅ SQL Piece: [EffectiveAt] = #2021-03-01 00:00:00.000000#
      │  └•[SET PARAM VALUE]
      │     Lusid.Portfolio.Holding: EffectiveAt
      │     └•DateTime ⬅ SQL Piece: #2021-03-01 00:00:00.000000#
      │        └•[LITERAL]
      │           literal_input: #2021-03-01 00:00:00.000000#
      ├•Parameter ⬅ SQL Piece: [AsAt] = #2021-03-08 00:00:00.000000#
      │  └•[SET PARAM VALUE]
      │     Lusid.Portfolio.Holding: AsAt
      │     └•DateTime ⬅ SQL Piece: #2021-03-08 00:00:00.000000#
      │        └•[LITERAL]
      │           literal_input: #2021-03-08 00:00:00.000000#
      └•Provider Definition: lusid_portfolio_holding
         ├•Table SQL Name: Lusid.Portfolio.Holding
         ├•Type: DataProvider
         ├•Category: Lusid
         ├•Description: Returns Lusid Portfolio / Holdings data
         ├•Documentation: [Not Available]
         ├•Last Ping At: 2021-06-13 14:46:50.179
         └•Fields:
            ├•as_at       

## Provider Columns

Columns are available as attributes on the provider instance and you can use get_columns() to get a list of the columns that live on this data provider. 

In [5]:
holdings.lusid_instrument_id

└•Text ⬅ SQL Piece: [LusidInstrumentId]
   └•[COLUMN INPUT]
      Lusid.Portfolio.Holding: LusidInstrumentId

In [6]:
for c in holdings.get_columns():
    print(f"Column {c.get_name():30} -> Resolves to SQL -> {c.get_sql()}")

Column cost_amount                    -> Resolves to SQL -> [CostAmount]
Column cost_amount_portfolio_currency -> Resolves to SQL -> [CostAmountPortfolioCurrency]
Column cost_currency                  -> Resolves to SQL -> [CostCurrency]
Column error                          -> Resolves to SQL -> [Error]
Column holding_prop                   -> Resolves to SQL -> [HoldingProp]
Column holding_type                   -> Resolves to SQL -> [HoldingType]
Column inst_prop                      -> Resolves to SQL -> [InstProp]
Column instrument_tag                 -> Resolves to SQL -> [InstrumentTag]
Column lusid_instrument_id            -> Resolves to SQL -> [LusidInstrumentId]
Column portfolio_code                 -> Resolves to SQL -> [PortfolioCode]
Column portfolio_scope                -> Resolves to SQL -> [PortfolioScope]
Column settled_units                  -> Resolves to SQL -> [SettledUnits]
Column sub_holding_key                -> Resolves to SQL -> [SubHoldingKey]
Column units   

## Select Statements

Given the above as a starting point we can now start to build the query. All queries must start by calling the `.select(...)` method on a provider class. 

Any input to select that is not an original column of the provider (i.e. function of columns or literals) must be supplied as a keyword argument where the argument name is the alias you'll give to the new column. 

In [7]:
selected = holdings.select('*')
print(f"For '*' there will be {len(selected.get_columns())} columns returned.")

selected = holdings.select('^')
print(f"For '^' there will be {len(selected.get_columns())} columns returned.")

selected = holdings.select(
    holdings.lusid_instrument_id, 
    holdings.cost_amount,
    holdings.cost_currency,
    holdings.settled_units,
    Millions=holdings.cost_amount*1e-6, # Function of provider column input
    SomeExchangeRate=1.5 # Literal input
)
print(f"With 4 cols selected + 2 aliases there will be {len(selected.get_columns())} columns returned.")

For '*' there will be 14 columns returned.
For '^' there will be 9 columns returned.
With 4 cols selected + 2 aliases there will be 6 columns returned.


We can use the `.print_sql()` method with any expression to see the SQL string that it resolves to. 

In [8]:
selected.print_sql()


select
  [LusidInstrumentId], [CostAmount], [CostCurrency], [SettledUnits], [CostAmount] * 1e-06 as [Millions], 1.5 as [SomeExchangeRate] 
from
  Lusid.Portfolio.Holding 
where
  [EffectiveAt] = #2021-03-01 00:00:00.000000# and [AsAt] = #2021-03-08 00:00:00.000000#


Once select is called we can then chain off other methods that represent other SQL clauses. For example we can add a `limit 5` as follows. 

In [9]:
qry = selected.limit(5)

Once you are happy with your query you can send it off to luminesce by call its go method. If an object has a go() method it should always be valid SQL. This method will send the query off, monitor its progress in the cell and then get the result back once finished as a pandas DataFrame. 

In [10]:
df = qry.go()
df

,LusidInstrumentId,CostAmount,CostCurrency,SettledUnits,Millions,SomeExchangeRate
0,LUID_JTQY6QFI,2.390760e+09,USD,1.200000e+07,2390.760000,1.5
1,LUID_J4G8D0DV,1.329858e+09,EUR,8.390274e+06,1329.858354,1.5
2,LUID_TMYK4GZV,7.197106e+10,JPY,1.098658e+07,71971.057729,1.5
3,LUID_WW55WKWV,-1.092849e+09,USD,-8.191471e+06,-1092.849359,1.5
4,LUID_7M3OX0BB,1.314567e+09,USD,1.085971e+07,1314.567397,1.5


## Column Expressions

You can build up complex expressions easily using normal python. N.B. that you must use the `&` operator with bracketed expressions and not `and` in a similar way to filtering pandas. This is because the `and` operator casts its result to a boolean and doesn't allow the sort of operator overloading we want to use here. 

The following expression gets holdings in the Finbourne-Examples scope with cost in USD

In [11]:
condition = (holdings.portfolio_scope == 'Finbourne-Examples') & (holdings.cost_currency == 'USD')
# Show the underlying DAG
condition

└•Boolean ⬅ SQL Piece: ([PortfolioScope] = 'Finbourne-Examples') and ([CostCurrency] = 'USD')
   └•[AND]
      ├•Boolean ⬅ SQL Piece: [PortfolioScope] = 'Finbourne-Examples'
      │  └•[EQUAL]
      │     ├•Text ⬅ SQL Piece: [PortfolioScope]
      │     │  └•[COLUMN INPUT]
      │     │     Lusid.Portfolio.Holding: PortfolioScope
      │     └•Text ⬅ SQL Piece: 'Finbourne-Examples'
      │        └•[LITERAL]
      │           literal_input: 'Finbourne-Examples'
      └•Boolean ⬅ SQL Piece: [CostCurrency] = 'USD'
         └•[EQUAL]
            ├•Text ⬅ SQL Piece: [CostCurrency]
            │  └•[COLUMN INPUT]
            │     Lusid.Portfolio.Holding: CostCurrency
            └•Text ⬅ SQL Piece: 'USD'
               └•[LITERAL]
                  literal_input: 'USD'

## Where Filter Expression

Apply the above condition in a where expression. 

Can also use `filter` instead of where. It resolves to the same SQL. 

In [12]:
qry = holdings.select(
    holdings.cost_currency,
    holdings.lusid_instrument_id,
    holdings.portfolio_scope,
    holdings.cost_amount_portfolio_currency
).where(
    condition
)

In [13]:
df = qry.go()
df.head()

,CostCurrency,LusidInstrumentId,PortfolioScope,CostAmountPortfolioCurrency
0,USD,LUID_JTQY6QFI,Finbourne-Examples,1.822715e+09
1,USD,LUID_WW55WKWV,Finbourne-Examples,-8.224518e+08
2,USD,LUID_7M3OX0BB,Finbourne-Examples,1.002226e+09
3,USD,LUID_1W5WYN3O,Finbourne-Examples,1.760183e+09
4,USD,CCY_USD,Finbourne-Examples,0.000000e+00


## Group by / Aggregation Expression and Order By

Aggregate total cost holdings by instrument ID, order by total cost and limit to top 5. 

The `.group_by()` method must be followed by the `.aggregate()` method and the aggregations must be named by supplying them as keyword arguments. Aggregations are built by calling methods on column expressions such as `.sum()` or `.mean()`.

The `.order_by()` method will accept any number of orderings and will sort in the method arg order. Orderings are created by calling the `.ascending()` or `.descending()` methods on a column expression. 

In [14]:
total_cost = holdings.cost_amount_portfolio_currency.sum()
total_cost

└•Decimal ⬅ SQL Piece: total([CostAmountPortfolioCurrency])
   └•[TOTAL]
      └•Decimal ⬅ SQL Piece: [CostAmountPortfolioCurrency]
         └•[COLUMN INPUT]
            Lusid.Portfolio.Holding: CostAmountPortfolioCurrency

In [15]:
ordering = total_cost.descending()
ordering

└•Ordering ⬅ SQL Piece: total([CostAmountPortfolioCurrency]) desc
   └•[DESC ORDER]
      └•Decimal ⬅ SQL Piece: total([CostAmountPortfolioCurrency])
         └•[TOTAL]
            └•Decimal ⬅ SQL Piece: [CostAmountPortfolioCurrency]
               └•[COLUMN INPUT]
                  Lusid.Portfolio.Holding: CostAmountPortfolioCurrency

In [16]:
top5_cost = holdings.select(
    holdings.cost_currency,
    holdings.lusid_instrument_id,
).where(
    condition
).group_by(
    holdings.lusid_instrument_id
).aggregate(
    TotalCostAmount=total_cost
).order_by(
    ordering
).limit(5)

In [17]:
df = top5_cost.go()
df

,CostCurrency,LusidInstrumentId,TotalCostAmount
0,USD,LUID_JTQY6QFI,1.822715e+09
1,USD,LUID_1W5WYN3O,1.760183e+09
2,USD,LUID_7M3OX0BB,1.002226e+09
3,USD,LUID_2VUE91BY,1.527824e+08
4,USD,LUID_XB11B4VW,1.134238e+08


## Having Filter Expression

Filter the above aggregation for only rows over one billion in cost

Can also use `filter` instead of having. It resolves to the same SQL. 

In [18]:
over_1billion = holdings.select(
    holdings.cost_currency,
    holdings.lusid_instrument_id,
).where(
    condition
).group_by(
    holdings.lusid_instrument_id
).aggregate(
    TotalCostAmount=total_cost
).having(
    total_cost > 1e9
)
over_1billion.go()

,CostCurrency,LusidInstrumentId,TotalCostAmount
0,USD,LUID_1W5WYN3O,1.760183e+09
1,USD,LUID_7M3OX0BB,1.002226e+09
2,USD,LUID_JTQY6QFI,1.822715e+09
